In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
import string
import re
import requests
from werkzeug.exceptions import HTTPException, NotFound
from datetime import datetime
!pip install meteostat
from meteostat import Hourly, Daily, Stations
from pprint import pprint
!pip install html-table-parser-python3
from html_table_parser.parser import HTMLTableParser

# Gathering all of the active players listed in bold, gathering their data into a dictionary, converting that into a dataframe, and filtering for only offensive players

In [ ]:
###     GTG     ###
ros_url = 'https://www.pro-football-reference.com/players/'
alpha = list(string.ascii_uppercase)
html = urlopen(ros_url)
stats_page = BeautifulSoup(html)

In [ ]:
###     GTG     ###
ros_url_list = []
for i in alpha:
  ros_url_list.append(ros_url + i)

In [ ]:
from numba.core.decorators import njit
###     GTG     ###
#SCRAPING ALL ACTIVE PLAYERS FROM PRO-FOOTBALL-REFERENCE FROM EVERY PAGE
#PARSING AND ASSIGNING SPECIFIC INFO INTO A DICTIONARY
players = []

for i in ros_url_list:
  stats_page = BeautifulSoup(urlopen(i))
  for j in stats_page.findAll('b'):
    url = [j.get('href') for j in j.find_all('a')]
    players.append({'player': j.getText().strip(')').split('(')[0].rstrip(), 'pos': (j.getText().strip(')').split('('))[-1], 'url': ' '.join(url)})

In [ ]:
###     GTG     ###
#TURNING DICTIONARY INTO DATAFRAME
player_df = pd.DataFrame.from_dict(players)

# Filtering player list for just offensive players, calling player specific url

In [ ]:
###     GTG     ###
#FILTER FOR QB, RB, WR, TE, K
#RESETTING INDEX AS ITS OWN DF
off_player = player_df[(player_df.pos == 'QB') | (player_df.pos == 'RB') | (player_df.pos == 'WR') | (player_df.pos == 'TE') | (player_df.pos == 'K')]
off_player.reset_index(drop=True, inplace=True)

In [ ]:
###     GTG     ###
#ADDS SPECIFIC PLAYER URL TO OFF_PLAYER DF
play_list = []
play_url = 'https://www.pro-football-reference.com'

for i in off_player['url']:
  play_list.append(play_url + i + '/gamelog/')
off_player['new_url'] = play_list

# Pull in Stadium data and add some missing information

In [ ]:
###     GTG     ###
#Call and sort DEF Data - DONT HAVE TO CALL MORE THAN ONCE WHEN FINALIZED
team_keys = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det', 'gnb', 'htx', 'clt', 'jax', 'kan', \
             'rai', 'sdg', 'ram', 'mia', 'min', 'nwe', 'nor', 'nyg', 'nyj', 'phi', 'pit', 'sfo', 'sea', 'tam', 'oti', 'was']
dicts = {}

for i in team_keys:
  dicts[i] = ['https://www.pro-football-reference.com/teams/' + i + '/' + str(j) + '.htm' for j in range(2002, 2023)]  ### UPDATE THIS BEFORE EVERY SEASON ###
  team_df = pd.DataFrame.from_dict(dicts)

for i in dicts:
  defs = pd.DataFrame()
  defs = defs.append([pd.DataFrame(pd.read_html(j)[1]).assign(Year=k + 2002) for k, j in enumerate(dicts[i])])

  lisp = [  ('Unnamed: 0_level_0',               'Week'),
            ('Unnamed: 1_level_0',                'Day'),
            ('Unnamed: 2_level_0',               'Date'),
            ('Unnamed: 3_level_0', 'Unnamed: 3_level_1'),
            ('Unnamed: 4_level_0', 'Unnamed: 4_level_1'),
            ('Unnamed: 5_level_0', 'Unnamed: 5_level_1'),
            ('Unnamed: 6_level_0',                 'OT'),
            ('Unnamed: 7_level_0',                'Rec'),
            ('Unnamed: 8_level_0', 'Unnamed: 8_level_1'),
            ('Unnamed: 9_level_0',                'Opp'),
            (             'Score',                 'Tm'),
            (             'Score',                'Opp'),
            (           'Defense',               '1stD'),
            (           'Defense',              'TotYd'),
            (           'Defense',              'PassY'),
            (           'Defense',              'RushY'),
            (           'Defense',                 'TO'),
            (              'Year', )]
  #col_list = []
  #for k in lisp:
    #if k in defs.columns:
     #col_list.append(k)
  #defs = defs.loc[:, col_list]

  #Add 'Year' and 'Tm' column in here somewhere
  defs.insert(8, 'Tm', i)

  defs.to_csv('/content/drive/MyDrive/NFL Data/Player CSV/DEF/' + i + '.csv')

**WHEN IMPORTING ANY CSV WITH MULTI-INDEX, YOU MUST IMPORT WITH HEADER=[0,1]...OR HOWEVER MANY ROWS OF HEADERS**

In [ ]:
###     GTG     ###

###     ADJUST FOR ALL DATA     ###
for i in off_player.index:
  play_pos = off_player['pos'][i]
  name = off_player['player'][i] + ' ' + off_player['pos'][i] + '.csv'
  filepath = '/content/drive/MyDrive/NFL Data/Test Folder/'

  #Pull Tables
  stats = pd.read_html(off_player['new_url'][i])[0]
  stats = pd.DataFrame(stats)

  #Drop Rows & Reindex
  stats.drop(stats[stats['Unnamed: 0_level_0']['Rk'] == 'Rk'].index, inplace=True)
  stats = stats[0:-1]
  stats.reset_index(drop=True, inplace=True)

  #Adjust to Only the Columns Needed
  lisp = [  ( 'Unnamed: 0_level_0',                 'Rk'),
            ( 'Unnamed: 1_level_0',               'Year'),
            ( 'Unnamed: 2_level_0',               'Date'),
            ( 'Unnamed: 3_level_0',                 'G#'),
            ( 'Unnamed: 4_level_0',               'Week'),
            ( 'Unnamed: 5_level_0',                'Age'),
            ( 'Unnamed: 6_level_0',                 'Tm'),
            ( 'Unnamed: 7_level_0', 'Unnamed: 7_level_1'),
            ( 'Unnamed: 8_level_0',                'Opp'),
            ( 'Unnamed: 9_level_0',             'Result'),
            ('Unnamed: 10_level_0',                 'GS'),
            (            'Passing',                'Att'),
            (            'Passing',                'Yds'),
            (            'Passing',                 'TD'),
            (            'Passing',                'Int'),
            (            'Passing',                 'Sk'),
            (            'Rushing',                'Att'),
            (            'Rushing',                'Yds'),
            (            'Rushing',                 'TD'),
            (          'Receiving',                'Tgt'),
            (          'Receiving',                'Rec'),
            (          'Receiving',                'Yds'),
            (          'Receiving',                 'TD'),
            (       'Kick Returns',                 'TD'),
            (               'Punt',                 'TD'),
            (            'Scoring',                '2PM'),
            (            'Scoring',                'XPA'),
            (            'Scoring',                'XPM'),
            (            'Scoring',                'FGA'),
            (            'Scoring',                'FGM'),
            (            'Fumbles',                 'FL')]
  col_list = []
  for i in lisp:
    if i in stats.columns:
     col_list.append(i)
  stats = stats.loc[:, col_list]

  stats['pos'] = play_pos

  #Save data
  if play_pos == 'QB':
    stats.to_csv('/content/drive/MyDrive/NFL Data/Player CSV/QB/' + name)
  elif play_pos == 'RB':
    stats.to_csv('/content/drive/MyDrive/NFL Data/Player CSV/RB/' + name)
  elif play_pos == 'WR':
    stats.to_csv('/content/drive/MyDrive/NFL Data/Player CSV/WR/' + name)
  elif play_pos == 'TE':
    stats.to_csv('/content/drive/MyDrive/NFL Data/Player CSV/TE/' + name)
  elif play_pos == 'K':
    stats.to_csv('/content/drive/MyDrive/NFL Data/Player CSV/K/' + name)